In [1]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=07a82a5993773b1248b55299f26458e99bb0edbf64d19871ecbc0354bfe1a83e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import re
from string import punctuation
import pymorphy2
from collections import Counter

In [17]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances

In [32]:
from tensorflow.keras.optimizers.legacy import Adam

In [3]:
morph = pymorphy2.MorphAnalyzer()

In [4]:
def preprocess(text):
    tokens = re.sub('#+', ' ', text.lower()).split()
    tokens = [token.strip(punctuation) for token in tokens]
    tokens = [token for token in tokens if token]
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]  # добавили лемматизацию
    return lemmas

In [43]:
def most_similar(word, embeddings):
    similar = [id2word[i] for i in
               cosine_distances(embeddings[word2id[word]].reshape(1, -1), embeddings).argsort()[0][:10]]
    return similar

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты:
- добавьте лемматизацию в предобработку (любым способом)  
- измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window используется не так и вам нужно это  изменить!)  

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
wiki = open('/content/drive/MyDrive/ВШЭ/Магистратура/NLP/wiki_data.txt').read().split('\n')

In [7]:
vocab = Counter()

for text in wiki:
    vocab.update(preprocess(text))

In [8]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

In [9]:
len(filtered_vocab)


12461

In [10]:
word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [11]:
id2word = {i:word for word, i in word2id.items()}

In [12]:
sentences = []

for text in wiki:
    tokens = preprocess(text)
    if not tokens:
        continue
    ids = [word2id[token] for token in tokens if token in word2id]
    sentences.append(ids)

In [13]:
vocab_size = len(id2word)

In [34]:
# cbow
def gen_batches_cbow(sentences, window=6, batch_size=512):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2]

                X_target.append(word)
                X_context.append(context)
                y.append(1)

                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window*2)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [39]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [40]:
# cbow
model.fit(gen_batches_cbow(sentences[:19000], window=6),
          validation_data=gen_batches_cbow(sentences[19000:],  window=6),
          batch_size=512,
          steps_per_epoch=2500,
          validation_steps=30,
          epochs=2)

Epoch 1/2
2500/2500 [==============================] - 157s 63ms/step - loss: 0.4174 - accuracy: 0.8116 - val_loss: 0.4087 - val_accuracy: 0.8215
Epoch 2/2
2500/2500 [==============================] - 159s 64ms/step - loss: 0.3577 - accuracy: 0.8486 - val_loss: 0.3202 - val_accuracy: 0.8687


In [41]:
embeddings = model.layers[2].get_weights()[0]

In [46]:
most_similar('электрон', embeddings)

['электрон',
 'параметрический',
 '11600',
 'sims',
 'acoela',
 'бодуэн',
 'уайетта',
 'азотный',
 'анненский',
 'федченко']

In [47]:
most_similar('свет', embeddings)

['свет',
 'сцена',
 'шоу',
 'основа',
 'этап',
 'играть',
 'успех',
 'переход',
 'угол',
 'человеческий']

In [48]:
most_similar('пять', embeddings)

['пять',
 '4',
 'один',
 'два',
 'шесть',
 'каждый',
 'состоять',
 'участник',
 'из',
 'выйти']

In [49]:
most_similar('дом', embeddings)

['дом', 'на', 'они', 'он', 'для', 'но', 'как', 'который', 'весь', 'о']

In [50]:
most_similar('север', embeddings)

['север',
 'запад',
 'юг',
 'северо-запад',
 'восток',
 'северо-восток',
 'юго-запад',
 'юго-восток',
 'полуостров',
 'берег']

In [57]:
# skip gram
def gen_batches_sg(sentences, window, batch_size=512):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window//2):i] + sent[i+1:i+window//2]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)

                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)

                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

In [58]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))


embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)

dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context],
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [59]:
model.fit(gen_batches_sg(sentences[:19000], window=12),
          validation_data=gen_batches_sg(sentences[19000:],  window=12),
          batch_size=512,
          steps_per_epoch=2500,
          validation_steps=30,
          epochs=2)

Epoch 1/2
2500/2500 [==============================] - 146s 58ms/step - loss: 0.5049 - accuracy: 0.7545 - val_loss: 0.5285 - val_accuracy: 0.7359
Epoch 2/2
2500/2500 [==============================] - 141s 57ms/step - loss: 0.4496 - accuracy: 0.7944 - val_loss: 0.4950 - val_accuracy: 0.7891


In [60]:
embeddings = model.layers[2].get_weights()[0]

In [61]:
most_similar('электрон', embeddings)

['электрон',
 'расформировать',
 'дубовский',
 'увеличить',
 'опг',
 'хор',
 'дер',
 'метрополитен',
 '13-й',
 'бен']

In [62]:
most_similar('свет', embeddings)

['свет',
 'налог',
 'дубовский',
 'словенский',
 'филиал',
 'стекло',
 'аббат',
 'типичный',
 'станица',
 'расформировать']

In [63]:
most_similar('пять', embeddings)

['пять',
 'культовый',
 'расформировать',
 'перо',
 'готовность',
 'рекорд',
 'реконструкция',
 'нос',
 'умеренный',
 'уметь']

In [64]:
most_similar('дом', embeddings)

['дом',
 'дивеевский',
 '2019',
 'осетия',
 '1926',
 'частный',
 'ликвидировать',
 'хозяйство',
 'трижды',
 'звонок']

In [65]:
most_similar('север', embeddings)

['север',
 'тоннель',
 '2017',
 '1906',
 'боб',
 'населённый',
 'одиночный',
 'флаг',
 'налог',
 'пит']

cbow явно справляется лучше при таких параметрах

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [66]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
import gensim

In [69]:
texts = [preprocess(text) for text in wiki]

In [70]:
w2v = gensim.models.Word2Vec(texts,
                             vector_size=350,
                             min_count=25,
                             max_vocab_size=14000,
                             window=6,
                             epochs=8,
                             sg=0,   # то есть cbow
                             hs=0,
                             negative=10,
                             sample=1e-4,
                             ns_exponent=0.7,
                             cbow_mean=1)  # так и по умолчанию

In [71]:
w2v.wv.most_similar('дом')

[('особняк', 0.789076030254364),
 ('квартира', 0.7624276876449585),
 ('гостиница', 0.7311580777168274),
 ('баня', 0.7230255007743835),
 ('жилой', 0.6762901544570923),
 ('комната', 0.6713009476661682),
 ('сад', 0.6647835373878479),
 ('дача', 0.6359246969223022),
 ('этаж', 0.6206921339035034),
 ('помещение', 0.5956938862800598)]

In [75]:
w2v.wv.most_similar('свет')

[('солнечный', 0.6414508819580078),
 ('лунный', 0.6412933468818665),
 ('луна', 0.6405735015869141),
 ('увидеть', 0.6109490394592285),
 ('солнце', 0.5867289304733276),
 ('тёмный', 0.5748834013938904),
 ('атмосфера', 0.5341873168945312),
 ('ангел', 0.5321698784828186),
 ('небо', 0.531994104385376),
 ('титан', 0.5265035629272461)]

In [76]:
w2v.wv.most_similar('север')

[('юг', 0.9361886382102966),
 ('запад', 0.8861985802650452),
 ('северо-запад', 0.8320236802101135),
 ('северо-восток', 0.8319498896598816),
 ('восток', 0.8177932500839233),
 ('юго-восток', 0.8039749264717102),
 ('северо-восточный', 0.7529443502426147),
 ('граничить', 0.7509049773216248),
 ('юго-запад', 0.7447509765625),
 ('полуостров', 0.7439894080162048)]

In [77]:
w2v.wv.most_similar('пять')

[('четыре', 0.8580895066261292),
 ('шесть', 0.8487910628318787),
 ('три', 0.8432971239089966),
 ('семь', 0.8291167616844177),
 ('девять', 0.8120986223220825),
 ('восемь', 0.7640061378479004),
 ('двадцать', 0.7547066807746887),
 ('два', 0.751947283744812),
 ('десять', 0.7401976585388184),
 ('тридцать', 0.599853515625)]

In [78]:
ft = gensim.models.FastText(texts, min_n=1, max_n=9,
                            sg=0, hs=0,
                            negative=10,
                            vector_size=350,
                            window=6,
                            min_count=10,
                            ns_exponent=0.7)

In [79]:
ft.wv.most_similar('дом')

[('эдом', 0.8002668619155884),
 ('чудом', 0.7529065012931824),
 ('домбе', 0.7527614235877991),
 ('дом»', 0.7181474566459656),
 ('домен', 0.6979605555534363),
 ('радом', 0.6827585101127625),
 ('домохозяин', 0.6637206673622131),
 ('домик', 0.6625744104385376),
 ('дом-музей', 0.6472657322883606),
 ('селдом', 0.642882227897644)]

In [80]:
ft.wv.most_similar('пять')

[('опять', 0.9005196690559387),
 ('девять', 0.8660160303115845),
 ('зять', 0.8567721843719482),
 ('десять', 0.8394466042518616),
 ('повять', 0.837236225605011),
 ('нырять', 0.8367499113082886),
 ('пятнадцать', 0.8358696699142456),
 ('«пять', 0.8278015851974487),
 ('стоять', 0.8243265151977539),
 ('фуять', 0.8208210468292236)]

In [81]:
ft.wv.most_similar('север')

[('северо-запад', 0.9081919193267822),
 ('северо-восток', 0.9027411341667175),
 ('североморск', 0.8719360828399658),
 ('северин', 0.8706892728805542),
 ('северянин', 0.8287167549133301),
 ('юго-восток', 0.8284784555435181),
 ('северо-западный', 0.8224183320999146),
 ('северо-восточный', 0.8192083239555359),
 ('североирландец', 0.8185510039329529),
 ('юго-запад', 0.8033374547958374)]

In [82]:
ft.wv.most_similar('пяльца')

[('пыльца', 0.9251169562339783),
 ('пицца', 0.8428845405578613),
 ('плеяда', 0.825131893157959),
 ('пальма', 0.8173341155052185),
 ('пульхра', 0.8097302317619324),
 ('пьяница', 0.7992439866065979),
 ('памела', 0.7834677696228027),
 ('паула', 0.7756851315498352),
 ('ланца', 0.7746432423591614),
 ('пчела', 0.7726569771766663)]

In [83]:
ft.wv.most_similar('свет')

[('самоцвет', 0.8253276348114014),
 ('повет', 0.7864234447479248),
 ('светило', 0.781858503818512),
 ('цвет', 0.7722879648208618),
 ('расцвет', 0.7644054293632507),
 ('светофор', 0.7637861967086792),
 ('рассвет', 0.7574058175086975),
 ('ответ', 0.745500385761261),
 ('просвет', 0.7418829798698425),
 ('соцветие', 0.7377620935440063)]

Хорошо работают!